In [1]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd

In [ ]:
# Scarica i dati giornalieri dell'S&P 500 e dello STOXX50
sp500_data = yf.download('^GSPC', start='2020-01-01', end='2024-01-01',interval='1d')
stoxx50_data = yf.download('^STOXX', start='2020-01-01', end='2024-01-01', interval='1d')

# Reimposta l'indice
sp500_data.reset_index(inplace=True)
stoxx50_data.reset_index(inplace=True)

In [ ]:
# Normalizza i dati in modo che entrambi partano da 100
sp500_data['Normalized'] = (sp500_data['Adj Close'] / sp500_data['Adj Close'].iloc[0]) * 100
stoxx50_data['Normalized'] = (stoxx50_data['Adj Close'] / stoxx50_data['Adj Close'].iloc[0]) * 100

In [ ]:
# Unisci i dati in un unico DataFrame
merged_data = pd.merge(sp500_data[['Date', 'Normalized']], stoxx50_data[['Date', 'Normalized']], on='Date', suffixes=('_SP500', '_STOXX'))

# Calcola la rolling correlation con una finestra di 30 giorni
merged_data['RollingCorr'] = merged_data['Normalized_SP500'].rolling(window=12).corr(merged_data['Normalized_STOXX'])

from scipy.stats import spearmanr
# Calcola la rank correlation con una finestra di 30 giorni
def rolling_spearmanr(x, y, window):
    return [spearmanr(x[i:i+window], y[i:i+window])[0] if len(x[i:i+window]) == window else None for i in range(len(x))]

merged_data['RankCorr'] = rolling_spearmanr(merged_data['Normalized_SP500'], merged_data['Normalized_STOXX'], 30)

In [ ]:
# Creazione dei frame per l'animazione
frames = []
dates = merged_data['Date']
for date in dates:
    data_until_date = merged_data[merged_data['Date'] <= date]
    frames.append(go.Frame(
        data=[
            go.Scatter(x=data_until_date['Date'], y=data_until_date['Normalized_SP500'], mode='lines', name='S&P 500'),
            go.Scatter(x=data_until_date['Date'], y=data_until_date['Normalized_STOXX'], mode='lines', name='STOXX'),
            go.Scatter(x=data_until_date['Date'], y=data_until_date['RollingCorr'], mode='lines', name='Rolling Correlation', yaxis='y2'),
            go.Scatter(x=data_until_date['Date'], y=data_until_date['RankCorr'], mode='lines', name='Rank Correlation', yaxis='y2')
        ],
        name=str(date)
    ))

In [ ]:
# Creazione della figura iniziale
fig = go.Figure(
    data=[
        go.Scatter(x=merged_data['Date'], y=merged_data['Normalized_SP500'], mode='lines', name='S&P 500'),
        go.Scatter(x=merged_data['Date'], y=merged_data['Normalized_STOXX'], mode='lines', name='STOXX'),
        go.Scatter(x=merged_data['Date'], y=merged_data['RollingCorr'], mode='lines', name='Rolling Correlation', yaxis='y2'),
        go.Scatter(x=data_until_date['Date'], y=data_until_date['RankCorr'], mode='lines', name='Rank Correlation', yaxis='y2')
    ],
    layout=go.Layout(
        title="S&P 500 and STOXX Stock Market Over Time (Normalized to 100) with Rolling Correlation",
        title_x=0.5,  # Centra il titolo
        yaxis=dict(title='Normalized Adjusted Close', autorange=True, domain=[0.4, 1]),  # Ridurre il dominio per ingrandire il grafico inferiore
        yaxis2=dict(title='Rolling Correlation', autorange=True, domain=[0, 0.35], zeroline=True, zerolinewidth=0.5, zerolinecolor='red'),  # Aumentare il dominio per ingrandire il grafico inferiore

        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label="Start",
                    method="animate",
                    args=[
                        None,
                        {"frame": {"duration": 25, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 0}, "loop": True}
                    ]
                )
            ]
        )],
        width=1200,  # Aumentare la larghezza della figura
        height=800  # Aumentare l'altezza della figura
    ),
    frames=frames
)

# Visualizzazione dell'animazione nel notebook
fig.show()

In [ ]:
# Calcola i ritorni giornalieri
sp500_data['Returns'] = sp500_data['Adj Close'].pct_change()
stoxx50_data['Returns'] = stoxx50_data['Adj Close'].pct_change()

# Rimuovi i dati NaN generati dal calcolo dei ritorni
sp500_data.dropna(inplace=True)
stoxx50_data.dropna(inplace=True)

In [ ]:
# Unisci i dati in un unico DataFrame
merged_data = pd.merge(sp500_data[['Date', 'Returns']], stoxx50_data[['Date', 'Returns']], on='Date', suffixes=('_SP500', '_STOXX'))

In [ ]:
# Funzione per calcolare la percentuale di osservazioni in ciascun quadrante
def calculate_quadrant_percentages(data):
    total = len(data)
    q1 = data[(data['Returns_SP500'] >= 0) & (data['Returns_STOXX'] >= 0)].shape[0] / total * 100
    q2 = data[(data['Returns_SP500'] < 0) & (data['Returns_STOXX'] >= 0)].shape[0] / total * 100
    q3 = data[(data['Returns_SP500'] < 0) & (data['Returns_STOXX'] < 0)].shape[0] / total * 100
    q4 = data[(data['Returns_SP500'] >= 0) & (data['Returns_STOXX'] < 0)].shape[0] / total * 100
    return q1, q2, q3, q4

In [ ]:
# Creazione dei frame per l'animazione
frames = []
dates = merged_data['Date']
for i, date in enumerate(dates):
    data_until_date = merged_data.iloc[:i+1]
    q1, q2, q3, q4 = calculate_quadrant_percentages(data_until_date)
    frames.append(go.Frame(
        data=[
            go.Scatter(x=data_until_date['Returns_SP500'], y=data_until_date['Returns_STOXX'], mode='markers', marker=dict(color='blue')),
            go.Scatter(x=[merged_data['Returns_SP500'].min(), merged_data['Returns_SP500'].max()], y=[0, 0], mode='lines', line=dict(color='black', dash='dash')),
            go.Scatter(x=[0, 0], y=[merged_data['Returns_STOXX'].min(), merged_data['Returns_STOXX'].max()], mode='lines', line=dict(color='black', dash='dash'))
        ],
        name=str(date),
        layout=go.Layout(
            annotations=[
                dict(x=0.05, y=0.05, text=f"Q1: {q1:.1f}%", showarrow=False, font=dict(size=12, color='black')),
                dict(x=-0.05, y=0.05, text=f"Q2: {q2:.1f}%", showarrow=False, font=dict(size=12, color='black')),
                dict(x=-0.05, y=-0.05, text=f"Q3: {q3:.1f}%", showarrow=False, font=dict(size=12, color='black')),
                dict(x=0.05, y=-0.05, text=f"Q4: {q4:.1f}%", showarrow=False, font=dict(size=12, color='black'))
            ]
        )
    ))

In [ ]:
# Creazione della figura iniziale
fig = go.Figure(
    data=[
        go.Scatter(x=[], y=[], mode='markers', marker=dict(color='blue')),
        go.Scatter(x=[merged_data['Returns_SP500'].min(), merged_data['Returns_SP500'].max()], y=[0, 0], mode='lines', line=dict(color='black', dash='dash')),
        go.Scatter(x=[0, 0], y=[merged_data['Returns_STOXX'].min(), merged_data['Returns_STOXX'].max()], mode='lines', line=dict(color='black', dash='dash'))
    ],
    layout=go.Layout(
        title="Scatter Plot of Daily Returns: S&P 500 vs STOXX",
        xaxis_title="S&P 500 Daily Returns",
        yaxis_title="STOXX Daily Returns",
        xaxis=dict(zeroline=True, zerolinewidth=0.0, zerolinecolor='gray'),
        yaxis=dict(zeroline=True, zerolinewidth=0.0, zerolinecolor='gray'),
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label="Start",
                    method="animate",
                    args=[
                        None,
                        {"frame": {"duration": 25, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 0}, "loop": True}
                    ]
                )
            ]
        )]
    ),
    frames=frames
)
fig.show()

In [ ]:
import numpy as np
import scipy.stats as stats
# Calcola i quantili teorici e campionari per il QQ plot
sp500_sorted = np.sort(sp500_data['Returns'])
stoxx50_sorted = np.sort(stoxx50_data['Returns'])

# Quantili teorici della distribuzione normale
theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, len(sp500_sorted)))

# Quantili campionari
sp500_quantiles = np.quantile(sp500_sorted, np.linspace(0.01, 0.99, len(sp500_sorted)))
stoxx50_quantiles = np.quantile(stoxx50_sorted, np.linspace(0.01, 0.99, len(stoxx50_sorted)))

In [ ]:
# Creazione del QQ plot
fig = go.Figure()

# Aggiungi i punti del QQ plot per S&P 500
fig.add_trace(go.Scatter(
    x=theoretical_quantiles,
    y=sp500_quantiles,
    mode='markers',
    marker=dict(color='blue'),
    name='S&P 500'
))

# Aggiungi i punti del QQ plot per STOXX50
fig.add_trace(go.Scatter(
    x=theoretical_quantiles,
    y=stoxx50_quantiles,
    mode='markers',
    marker=dict(color='red'),
    name='STOXX50'
))

# Aggiungi la linea y=x
fig.add_trace(go.Scatter(
    mode='lines',
    line=dict(color='black', dash='dash'),
    name='y = x'
))

# Configura il layout del grafico
fig.update_layout(
    title="QQ Plot: S&P 500 vs STOXX50 Daily Returns",
    xaxis_title="Theoretical Quantiles",
    yaxis_title="Sample Quantiles",
    xaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='gray'),
    yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='gray')
)

# Visualizzazione del grafico
fig.show()